## Chapter 9: Recommender
### Ex4: Beauty
Dataset: Beauty_5.json.json
Read more about dataset: http://jmcauley.ucsd.edu/data/amazon/
(http://jmcauley.ucsd.edu/data/amazon/)
Requirement:
Read dataset
Pre-process data
Use "asin" (ProductID), "reviewerID" and overall (User's reviews for each product - rating) to
build model to predict overalls => Give recommendation for users.

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
!tar -xvf spark-2.4.0-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"
import findspark
findspark.init()

spark-2.4.0-bin-hadoop2.7/
spark-2.4.0-bin-hadoop2.7/python/
spark-2.4.0-bin-hadoop2.7/python/setup.cfg
spark-2.4.0-bin-hadoop2.7/python/pyspark/
spark-2.4.0-bin-hadoop2.7/python/pyspark/resultiterable.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/python/
spark-2.4.0-bin-hadoop2.7/python/pyspark/python/pyspark/
spark-2.4.0-bin-hadoop2.7/python/pyspark/python/pyspark/shell.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/heapq3.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/join.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/version.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/rdd.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/java_gateway.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/find_spark_home.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/_globals.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/worker.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/accumulators.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/mllib/
spark-2.4.0-bin-hadoop2.7/python/pyspark/mllib/feature.py
spark-2.4.0-bin-hadoop2.7/python/pyspark

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [3]:
%cd '/content/gdrive/My Drive/LDS9_K273_ONLINE_Đinh Viết Trung'

/content/gdrive/My Drive/LDS9_K273_ONLINE_Đinh Viết Trung


In [4]:
# import libraries
from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from datetime import datetime
from pyspark.sql.functions import mean, stddev, col, log
from pyspark.sql.functions import to_date, dayofweek, to_timestamp
from pyspark.sql import types 
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DateType
from pyspark.sql.functions import year, month
from pyspark.sql.functions import dayofmonth, weekofyear
from pyspark.sql.functions import split, explode
from pyspark.sql.functions import coalesce, first, lit
from pyspark.ml.feature import Binarizer
from pyspark.ml.feature import Bucketizer
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.sql.functions import regexp_extract, col
from pyspark.sql.functions import datediff
from pyspark.sql.functions import when

In [5]:
from pyspark import SparkContext
sc = SparkContext()

In [6]:
spark = SparkSession(sc)

In [7]:
data = spark.read.json("data/Beauty_5.json")
data.show(5,truncate=True)


+----------+-------+-------+--------------------+-----------+--------------+------------+--------------------+--------------+
|      asin|helpful|overall|          reviewText| reviewTime|    reviewerID|reviewerName|             summary|unixReviewTime|
+----------+-------+-------+--------------------+-----------+--------------+------------+--------------------+--------------+
|7806397051| [3, 4]|    1.0|Very oily and cre...|01 30, 2014|A1YJEY40YUW4SE|      Andrea|Don't waste your ...|    1391040000|
|7806397051| [1, 1]|    3.0|This palette was ...|04 18, 2014| A60XNB876KYML|  Jessica H.|         OK Palette!|    1397779200|
|7806397051| [0, 1]|    4.0|The texture of th...| 09 6, 2013|A3G6XNM240RMWA|       Karen|       great quality|    1378425600|
|7806397051| [2, 2]|    2.0|I really can't te...| 12 8, 2013|A1PQFP6SAJ6D80|       Norah|Do not work on my...|    1386460800|
|7806397051| [0, 0]|    3.0|It was a little s...|10 19, 2013|A38FVHZTNQ271F|   Nova Amor|          It's okay.|    1382

In [8]:
data_sub = data.select(['asin', 'overall', 'reviewerID'])
data_sub.count()

198502

In [9]:
from pyspark.sql.functions import col, udf
from pyspark.sql.functions import isnan, when, count, col
data_sub.show(5, truncate=True)

+----------+-------+--------------+
|      asin|overall|    reviewerID|
+----------+-------+--------------+
|7806397051|    1.0|A1YJEY40YUW4SE|
|7806397051|    3.0| A60XNB876KYML|
|7806397051|    4.0|A3G6XNM240RMWA|
|7806397051|    2.0|A1PQFP6SAJ6D80|
|7806397051|    3.0|A38FVHZTNQ271F|
+----------+-------+--------------+
only showing top 5 rows



In [10]:
data_sub.select([count(when(col(c).isNull(), c)).alias(c) for c in
 data_sub.columns]).toPandas().T


,0
asin,0
overall,0
reviewerID,0


In [11]:
# Distinct users and movies
users = data_sub.select("reviewerID").distinct().count()
products = data_sub.select("asin").distinct().count()
numerator = data_sub.count()

In [12]:
display(numerator, users, products)


198502

22363

12101

In [13]:
# Number of ratings matrix could contain if no empty cells
denominator = users * products
denominator


270614663

In [14]:
#Calculating sparsity
sparsity = 1 - (numerator*1.0 / denominator)
print ("Sparsity: "), sparsity


Sparsity: 


(None, 0.9992664772935825)

In [15]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
# Converting String to index
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.functions import col


In [16]:
# Create an indexer
indexer = StringIndexer(inputCol='asin', 
 outputCol='asin_idx')
# Indexer identifies categories in the data
indexer_model = indexer.fit(data_sub)
# Indexer creates a new column with numeric index values
data_indexed = indexer_model.transform(data_sub)
# Repeat the process for the other categorical feature
indexer1 = StringIndexer(inputCol='reviewerID', 
 outputCol='reviewerID_idx')
indexer1_model = indexer1.fit(data_indexed)
data_indexed = indexer1_model.transform(data_indexed)

data_indexed.show(5, truncate=True)


+----------+-------+--------------+--------+--------------+
|      asin|overall|    reviewerID|asin_idx|reviewerID_idx|
+----------+-------+--------------+--------+--------------+
|7806397051|    1.0|A1YJEY40YUW4SE|  6959.0|       18008.0|
|7806397051|    3.0| A60XNB876KYML|  6959.0|       10825.0|
|7806397051|    4.0|A3G6XNM240RMWA|  6959.0|        5924.0|
|7806397051|    2.0|A1PQFP6SAJ6D80|  6959.0|       12357.0|
|7806397051|    3.0|A38FVHZTNQ271F|  6959.0|        6087.0|
+----------+-------+--------------+--------+--------------+
only showing top 5 rows



In [17]:
data_indexed.select([count(when(col(c).isNull(), c)).alias(c) for c in
 data_indexed.columns]).toPandas().T

,0
asin,0
overall,0
reviewerID,0
asin_idx,0
reviewerID_idx,0


In [18]:
# Smaller dataset so we will use 0.8 / 0.2
(training, test) = data_indexed.randomSplit([0.8, 0.2])


In [19]:
# Creating ALS model and fitting data
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
als = ALS(maxIter=5, 
 regParam=0.09, 
 rank = 25,
 userCol="reviewerID_idx", 
 itemCol="asin_idx", 
 ratingCol="overall", 
 coldStartStrategy="drop",
 nonnegative=True)
model = als.fit(training)


In [20]:
predictions = model.transform(test)
predictions.select(["asin_idx", "reviewerID_idx", 
 "overall", "prediction"]).show(5)


+--------+--------------+-------+----------+
|asin_idx|reviewerID_idx|overall|prediction|
+--------+--------------+-------+----------+
|   148.0|        5258.0|    3.0| 3.6192174|
|   148.0|         271.0|    5.0|  4.238535|
|   148.0|         222.0|    4.0|  4.004179|
|   148.0|       19062.0|    5.0| 4.0239897|
|   148.0|          41.0|    4.0|  4.566919|
+--------+--------------+-------+----------+
only showing top 5 rows



In [21]:
evaluator = RegressionEvaluator(metricName="rmse", 
 labelCol="overall",
 predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))


Root-mean-square error = 1.3479188605119696


In [22]:
#RMSE nhỏ => tốt

In [23]:
# Providing Recommendations: for all users

# get 20 recommendations which have highest rating.
user_recs = model.recommendForAllUsers(20)

In [24]:
for user in user_recs.head(5):
    print(user)
    print("\n")


Row(reviewerID_idx=1580, recommendations=[Row(asin_idx=5321, rating=7.8450164794921875), Row(asin_idx=5813, rating=7.732507705688477), Row(asin_idx=9318, rating=7.7067718505859375), Row(asin_idx=7937, rating=7.614767551422119), Row(asin_idx=10469, rating=7.570760726928711), Row(asin_idx=12013, rating=7.521965503692627), Row(asin_idx=9665, rating=7.489423751831055), Row(asin_idx=5329, rating=7.466183185577393), Row(asin_idx=8386, rating=7.46504545211792), Row(asin_idx=3527, rating=7.41293478012085), Row(asin_idx=11227, rating=7.407604217529297), Row(asin_idx=9996, rating=7.382834434509277), Row(asin_idx=11243, rating=7.368070602416992), Row(asin_idx=10376, rating=7.366779327392578), Row(asin_idx=5132, rating=7.361858367919922), Row(asin_idx=8232, rating=7.355746269226074), Row(asin_idx=10483, rating=7.354709148406982), Row(asin_idx=4632, rating=7.320071220397949), Row(asin_idx=11282, rating=7.314446926116943), Row(asin_idx=10361, rating=7.282055377960205)])


Row(reviewerID_idx=4900, re

### Converting back to string form


In [25]:
import pandas as pd
recs=model.recommendForAllUsers(10).toPandas()

nrecs=recs.recommendations.apply(pd.Series) \
 .merge(recs, right_index = True, left_index = True) \
 .drop(["recommendations"], axis = 1) \
 .melt(id_vars = ['reviewerID_idx'], value_name = "recommendation") \
 .drop("variable", axis = 1) \
 .dropna() 
nrecs=nrecs.sort_values('reviewerID_idx')


nrecs=pd.concat([nrecs['recommendation']\
 .apply(pd.Series), nrecs['reviewerID_idx']], axis = 1)

nrecs.columns = [ 
 'ProductID_index',
 'Rating',
 'UserID_index' 
 ]


In [26]:
md=data_indexed.select(['reviewerID', 'reviewerID_idx', 'asin', 'asin_idx'])
md=md.toPandas()

dict1 =dict(zip(md['reviewerID_idx'],md['reviewerID']))
dict2=dict(zip(md['asin_idx'],md['asin']))

nrecs['reviewerID']=nrecs['UserID_index'].map(dict1)
nrecs['asin']=nrecs['ProductID_index'].map(dict2)
nrecs=nrecs.sort_values('reviewerID')
nrecs.reset_index(drop=True, inplace=True)

new=nrecs[['reviewerID','asin','Rating']]
new['recommendations'] = list(zip(new.asin, new.Rating))
res=new[['reviewerID','recommendations']] 

res_new=res['recommendations'].groupby([res.reviewerID])\
 .apply(list).reset_index()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [27]:
res_new

,reviewerID,recommendations
0,A00414041RD0BXM6WK0GX,"[(B00H8JPMX6, 4.927905559539795), (B000VOHH56,..."
1,A00473363TJ8YSZ3YAGG9,"[(B0009OAHQ4, 4.026127815246582), (B001H3JQ0E,..."
2,A00700212KB3K0MVESPIY,"[(B00161IKD6, 6.788311004638672), (B00H8JPMX6,..."
3,A0078719IR14X3NNUG0F,"[(B004OJIZO6, 8.449787139892578), (B0020MM8WO,..."
4,A01198201H0E3GHV2Z17I,"[(B00GYN9A08, 6.230189323425293), (B00HRGBSYW,..."
...,...,...
22353,AZZNK89PXD006,"[(B0016B7QAY, 4.036824703216553), (B00H8JPMX6,..."
22354,AZZQXL8VDCFTV,"[(B00CBYAP0S, 6.028794288635254), (B001C9QET4,..."
22355,AZZT1ERHBSNQ8,"[(B001C9QET4, 6.586795330047607), (B000ZECIDS,..."
22356,AZZU6NXB8YJN9,"[(B000P8559S, 6.017800331115723), (B00161IKD6,..."


In [30]:
res_new.to_csv("Chapter_9/Exercise4/beauty.csv")